In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/abc/all_fish/

/content/drive/MyDrive/abc/all_fish


In [ ]:
#!unzip NA_Fish_Dataset.zip

In [4]:
os.listdir("NEW")  

['Horse Mackerel',
 'european sea sturgeon',
 'fighter fish',
 'Gold fish',
 'jalebi fish',
 'Red hand fish',
 'anglerfish',
 'cat fish',
 'Gilt Head Bream',
 'Black Sea Sprat',
 'Red Mullet',
 'tank cleaner fish',
 'sea horse',
 'smalltooth fish',
 'Shrimp',
 'Sea Bass',
 'Red Sea Bream',
 'tequils splitfin',
 'sakhalin sturgeon',
 'Striped Red Mullet',
 'Trout']

In [5]:
!ls NEW/

 anglerfish		 'Horse Mackerel'     'sea horse'
'Black Sea Sprat'	 'jalebi fish'	       Shrimp
'cat fish'		 'Red hand fish'      'smalltooth fish'
'european sea sturgeon'  'Red Mullet'	      'Striped Red Mullet'
'fighter fish'		 'Red Sea Bream'      'tank cleaner fish'
'Gilt Head Bream'	 'sakhalin sturgeon'  'tequils splitfin'
'Gold fish'		 'Sea Bass'	       Trout


In [6]:
for i in os.listdir("NEW"):
  print(i,len(os.listdir("NEW/"+i)))

Horse Mackerel 50
european sea sturgeon 50
fighter fish 44
Gold fish 212
jalebi fish 10
Red hand fish 50
anglerfish 58
cat fish 24
Gilt Head Bream 50
Black Sea Sprat 46
Red Mullet 50
tank cleaner fish 16
sea horse 23
smalltooth fish 51
Shrimp 50
Sea Bass 50
Red Sea Bream 50
tequils splitfin 56
sakhalin sturgeon 54
Striped Red Mullet 50
Trout 30


In [8]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass
for i in os.listdir("NEW"):
  try:
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
  except:
    pass
  for j in os.listdir("NEW/"+i)[:35]:
    os.rename("NEW/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("NEW/"+i)[:15]:
    os.rename("NEW/"+i+"/"+j,"test/"+i+"/"+j)

In [9]:
def img_Data(dir_path,target_size,batch,class_lst,preprocessing,):
  if preprocessing:
    gen_object =ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object =ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                          target_size= target_size,
                                          batch_size=batch,
                                          class_mode='sparse',
                                          classes=class_lst,
                                          shuffle=True))

In [10]:
train_data_gen =img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen =img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 659 images belonging to 21 classes.
Found 209 images belonging to 21 classes.


In [12]:
base_model=tf.keras.applications.mobilenet_v2.MobileNetV2(
              input_shape=(224,224,3),
              alpha=1.0,
              include_top=False,
              weights='imagenet',
              input_tensor=None,
              pooling=None,
              classes=1000,
              classifier_activation='softmax')

In [13]:
base_model.trainable=False

In [14]:
model=tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1024,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
elst=callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck=callbacks.ModelCheckpoint('.mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [16]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
2/2 [==============================] - 164s 125s/step - loss: 6.3479 - accuracy: 0.0774 - val_loss: 2.6434 - val_accuracy: 0.4306
Epoch 2/10
2/2 [==============================] - 53s 17s/step - loss: 2.2106 - accuracy: 0.5873 - val_loss: 0.9364 - val_accuracy: 0.8421
Epoch 3/10
2/2 [==============================] - 60s 26s/step - loss: 0.8041 - accuracy: 0.9014 - val_loss: 0.6056 - val_accuracy: 0.8517
Epoch 4/10
2/2 [==============================] - 51s 17s/step - loss: 0.4269 - accuracy: 0.9120 - val_loss: 0.4371 - val_accuracy: 0.8804
Epoch 5/10
2/2 [==============================] - 51s 17s/step - loss: 0.2621 - accuracy: 0.9287 - val_loss: 0.3493 - val_accuracy: 0.8947
Epoch 6/10
2/2 [==============================] - 59s 48s/step - loss: 0.1866 - accuracy: 0.9363 - val_loss: 0.2677 - val_accuracy: 0.8756
Epoch 7/10
2/2 [==============================] - 60s 25s/step - loss: 0.1347 - accuracy: 0.9621 - val_loss: 0.2760 - val_accuracy: 0.8947
Epoch 8/10
2/2 [=========